<a href="https://colab.research.google.com/github/amontenegrot/COVID19/blob/main/COVID19_Colombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar datos

Datos oficiales:

[Mundo](https://ourworldindata.org/coronavirus-source-data)

[Colombia](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx)

[Bogotá](http://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/enfermedades-trasmisibles/covid19/?unapproved=410&moderation-hash=b01afdf1c01305df1f6b8a6a02826da5#comment-410)

In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
#import pycountry

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
from pylab import *
from scipy.optimize import curve_fit
from scipy.integrate import odeint

dfGlobal = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_cases.csv')
dfGlobalMuertes = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_deaths.csv')
dfCol = pd.read_csv('https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (14,22) have mixed types.Specify dtype option on import or set low_memory=False.



In [2]:
def Mayus(cadena):
  if cadena == 'Bogotá D.C.':
    return str(cadena)
  elif cadena == 'Cartagena D.T. y C.':
    return str(cadena)
  elif cadena == 'Barranquilla D.E.':
    return str(cadena)
  elif cadena == 'Santa Marta D.T. y C.':
    return str(cadena)
  elif cadena == 'Buenaventura D.E.':
    return str(cadena)
  elif cadena == 'Santa marta D.T. y C.':
    return str(cadena)
  elif cadena == 'Archipiélago de San Andrés Providencia y Santa Catalina':
    return 'San Andrés, Providencia y Santa Catalina'
  elif cadena == 'Hospital UCI':
    return str(cadena)
  #elif cadena == 'F':
    #return 'Mujer'
  #elif cadena == 'M':
    #return 'Hombre'
  else:
    str(cadena)
    return cadena.capitalize()

#se busca en una cadena y lo que se busca se reemplaza
def Reemplazar(cadena, buscar, reemplazo):
  buscar = str(buscar)
  reemplazo = str(reemplazo)
  return cadena.replace(buscar, reemplazo)

#Preprocesamiento

In [3]:
dfCol.dtypes

fecha reporte web                object
ID de caso                        int64
Fecha de notificación            object
Código DIVIPOLA departamento      int64
Nombre departamento              object
Código DIVIPOLA municipio         int64
Nombre municipio                 object
Edad                              int64
Unidad de medida de edad          int64
Sexo                             object
Tipo de contagio                 object
Ubicación del caso               object
Estado                           object
Código ISO del país             float64
Nombre del país                  object
Recuperado                       object
Fecha de inicio de síntomas      object
Fecha de muerte                  object
Fecha de diagnóstico             object
Fecha de recuperación            object
Tipo de recuperación             object
Pertenencia étnica              float64
Nombre del grupo étnico          object
dtype: object

In [4]:
dfCol.columns

Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país',
       'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación', 'Pertenencia étnica',
       'Nombre del grupo étnico'],
      dtype='object')

In [5]:
dfCol.tail()

,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,Tipo de contagio,Ubicación del caso,Estado,Código ISO del país,Nombre del país,Recuperado,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico
1594492,27/12/2020 0:00:00,1594533,24/12/2020 0:00:00,11,BOGOTA,11001,BOGOTA,50,1,F,En estudio,Casa,Leve,NaN,NaN,Activo,23/12/2020 0:00:00,NaN,26/12/2020 0:00:00,NaN,NaN,NaN,NaN
1594493,27/12/2020 0:00:00,1594534,24/12/2020 0:00:00,11,BOGOTA,11001,BOGOTA,25,1,M,En estudio,Casa,Leve,NaN,NaN,Activo,23/12/2020 0:00:00,NaN,23/12/2020 0:00:00,NaN,NaN,NaN,NaN
1594494,27/12/2020 0:00:00,1594535,24/12/2020 0:00:00,11,BOGOTA,11001,BOGOTA,55,1,M,En estudio,Casa,Leve,NaN,NaN,Activo,23/12/2020 0:00:00,NaN,23/12/2020 0:00:00,NaN,NaN,NaN,NaN
1594495,27/12/2020 0:00:00,1594536,25/12/2020 0:00:00,11,BOGOTA,11001,BOGOTA,80,1,M,En estudio,Hospital UCI,Grave,NaN,NaN,Activo,20/12/2020 0:00:00,NaN,24/12/2020 0:00:00,NaN,NaN,NaN,NaN
1594496,27/12/2020 0:00:00,1594537,23/12/2020 0:00:00,11,BOGOTA,11001,BOGOTA,13,1,F,En estudio,Casa,Leve,NaN,NaN,Activo,19/12/2020 0:00:00,NaN,22/12/2020 0:00:00,NaN,NaN,NaN,NaN


In [6]:
#Eliminar columnas innecesarias
dfCol = dfCol.drop(['fecha reporte web'], axis=1)
dfCol = dfCol.drop(['Código DIVIPOLA departamento'], axis=1)
dfCol = dfCol.drop(['Código DIVIPOLA municipio'], axis=1)
dfCol = dfCol.drop(['Unidad de medida de edad'], axis=1)
dfCol = dfCol.drop(['Tipo de contagio'], axis=1)
dfCol = dfCol.drop(['Código ISO del país'], axis=1)
dfCol = dfCol.drop(['Nombre del país'], axis=1)
dfCol = dfCol.drop(['Recuperado'], axis=1)
dfCol = dfCol.drop(['Fecha de inicio de síntomas'], axis=1)
dfCol = dfCol.drop(['Fecha de muerte'], axis=1)
dfCol = dfCol.drop(['Fecha de diagnóstico'], axis=1)
dfCol = dfCol.drop(['Fecha de recuperación'], axis=1)
dfCol = dfCol.drop(['Tipo de recuperación'], axis=1)
dfCol = dfCol.drop(['Pertenencia étnica'], axis=1)
dfCol = dfCol.drop(['Nombre del grupo étnico'], axis=1)

In [7]:
dfCol.columns

Index(['ID de caso', 'Fecha de notificación', 'Nombre departamento',
       'Nombre municipio', 'Edad', 'Sexo', 'Ubicación del caso', 'Estado'],
      dtype='object')

In [8]:
#Renombrar columnas
dfCol = dfCol.rename({'Fecha de notificación':'Fecha',
                      'Nombre municipio':'Ciudad',
                      'Ubicación del caso':'Ubicacion',
                      'Nombre departamento':'Departamento'}, axis=1)
#Nueva columna con número de casos
dfCol['Casos'] = 1

In [9]:
dfCol.tail()

,ID de caso,Fecha,Departamento,Ciudad,Edad,Sexo,Ubicacion,Estado,Casos
1594492,1594533,24/12/2020 0:00:00,BOGOTA,BOGOTA,50,F,Casa,Leve,1
1594493,1594534,24/12/2020 0:00:00,BOGOTA,BOGOTA,25,M,Casa,Leve,1
1594494,1594535,24/12/2020 0:00:00,BOGOTA,BOGOTA,55,M,Casa,Leve,1
1594495,1594536,25/12/2020 0:00:00,BOGOTA,BOGOTA,80,M,Hospital UCI,Grave,1
1594496,1594537,23/12/2020 0:00:00,BOGOTA,BOGOTA,13,F,Casa,Leve,1


In [10]:
#Reemplazar errores de tipeo en cada columna (mayúscula inicial)
dfCol['Ciudad'] = dfCol['Ciudad'].apply(lambda x: Mayus(x))
dfCol['Departamento'] = dfCol['Departamento'].apply(lambda x: Mayus(x))
dfCol['Ubicacion'] = dfCol['Ubicacion'].apply(lambda x: str(x)) #Volver todos los valores tipo str
dfCol['Ubicacion'] = dfCol['Ubicacion'].apply(lambda x: Mayus(x))
dfCol['Sexo'] = dfCol['Sexo'].apply(lambda x: Mayus(x))
#dfCol['Tipo'] = dfCol['Tipo'].apply(lambda x: Mayus(x))
dfCol['Estado'] = dfCol['Estado'].apply(lambda x: str(x)) #Volver todos los valores tipo str
dfCol['Estado'] = dfCol['Estado'].apply(lambda x: Mayus(x))

In [11]:
dfCol['Estado'].value_counts()

Leve         1518154
Fallecido      42171
Moderado       25352
Nan             5243
Grave           3577
Name: Estado, dtype: int64

In [12]:
dfCol['Fecha'] = pd.to_datetime(dfCol['Fecha']) #Conversión a formato de 'fecha'

In [13]:
dfCol.dtypes

ID de caso               int64
Fecha           datetime64[ns]
Departamento            object
Ciudad                  object
Edad                     int64
Sexo                    object
Ubicacion               object
Estado                  object
Casos                    int64
dtype: object

In [14]:
dfCol.head()

,ID de caso,Fecha,Departamento,Ciudad,Edad,Sexo,Ubicacion,Estado,Casos
0,1,2020-02-03,Bogota,Bogota,19,F,Casa,Leve,1
1,2,2020-06-03,Valle,Buga,34,M,Casa,Leve,1
2,3,2020-07-03,Antioquia,Medellin,50,F,Casa,Leve,1
3,4,2020-09-03,Antioquia,Medellin,55,M,Casa,Leve,1
4,5,2020-09-03,Antioquia,Medellin,25,M,Casa,Leve,1


#Análisis

##Casos totales

In [15]:
casos = dfCol['Casos'].sum()
print('Total de casos a nivel nacional:', casos)

Total de casos a nivel nacional: 1594497


##Casos por sexo

In [16]:
#Transformar el objeto 'value_counts' en DataFrame
dfSexo = dfCol['Sexo']
dfSexo = dfSexo.value_counts()
dfSexo = dfSexo.rename_axis('Sexo')
dfSexo = dfSexo.reset_index(name='Numero')

In [17]:
dfSexo['Porcentaje'] = dfSexo['Numero'].apply(lambda x: (x/casos)*100)

dfSexo

,Sexo,Numero,Porcentaje
0,F,806372,50.572187
1,M,788125,49.427813


In [18]:
fig = go.Figure(data=[go.Pie(labels=dfSexo['Sexo'], values=dfSexo['Numero'], hole=.5)])
fig.show()

##Casos por departamentos

In [19]:
#Transformar el objeto 'value_counts' en DataFrame
dfDept = dfCol['Departamento']
dfDept = dfDept.value_counts()
dfDept = dfDept.rename_axis('Departamento')
dfDept = dfDept.reset_index(name='Numero')

In [20]:
dfDept['Porcentaje'] = dfDept['Numero'].apply(lambda x: (x/casos)*100)

dfDept

,Departamento,Numero,Porcentaje
0,Bogota,452940,28.406450
1,Antioquia,255845,16.045499
2,Valle,133536,8.374804
3,Santander,65278,4.093956
4,Cundinamarca,64803,4.064166
5,Barranquilla,54725,3.432117
6,Tolima,42407,2.659585
7,Cartagena,42318,2.654003
8,Norte santander,38993,2.445473
9,Atlantico,37024,2.321986


In [21]:
fig = px.scatter(x=dfDept['Departamento'], y=dfDept['Numero'],
                 size=dfDept['Porcentaje'], #color=dfDept['Departamento'],
                 hover_name=dfDept['Departamento'], size_max=50,
                 title='Número de casos por departamento o distrito especial', 
                 labels=list(dfDept['Departamento']))
fig.show()

##Ubicación de los casos

In [22]:
#Transformar el objeto 'value_counts' en DataFrame
dfUbicacion = dfCol['Ubicacion']
dfUbicacion = dfUbicacion.value_counts()
dfUbicacion = dfUbicacion.rename_axis('Ubicacion')
dfUbicacion = dfUbicacion.reset_index(name='Numero')

dfUbicacion

,Ubicacion,Numero
0,Casa,1518154
1,Fallecido,42171
2,Hospital,25352
3,Nan,5243
4,Hospital UCI,3577
